In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from keras.applications import MobileNet

In [3]:
img_rows, img_cols = 224, 224

MobileNet = MobileNet(weights= 'imagenet', 
                      include_top = False,
                      input_shape = (img_rows , img_cols , 3))

In [4]:
for layer in MobileNet.layers:
  layer.trainable = False

In [5]:
for i,layer in enumerate(MobileNet.layers):
  print(str(i) + " " + layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 BatchNormalization False
3 ReLU False
4 DepthwiseConv2D False
5 BatchNormalization False
6 ReLU False
7 Conv2D False
8 BatchNormalization False
9 ReLU False
10 ZeroPadding2D False
11 DepthwiseConv2D False
12 BatchNormalization False
13 ReLU False
14 Conv2D False
15 BatchNormalization False
16 ReLU False
17 DepthwiseConv2D False
18 BatchNormalization False
19 ReLU False
20 Conv2D False
21 BatchNormalization False
22 ReLU False
23 ZeroPadding2D False
24 DepthwiseConv2D False
25 BatchNormalization False
26 ReLU False
27 Conv2D False
28 BatchNormalization False
29 ReLU False
30 DepthwiseConv2D False
31 BatchNormalization False
32 ReLU False
33 Conv2D False
34 BatchNormalization False
35 ReLU False
36 ZeroPadding2D False
37 DepthwiseConv2D False
38 BatchNormalization False
39 ReLU False
40 Conv2D False
41 BatchNormalization False
42 ReLU False
43 DepthwiseConv2D False
44 BatchNormalization False
45 ReLU False
46 Conv2D False
47 BatchNormalization False
48

In [15]:
def addTopModelMobileNet(bottom_model, num_classes = 2):
  top_model = bottom_model.output
  top_model = GlobalAveragePooling2D()(top_model)
  top_model = Dense(units = 512, activation = 'relu')(top_model)
  top_model = Dense(units = 512, activation = 'relu')(top_model)
  top_model = Dense(units = 512, activation = 'relu')(top_model)
  top_model = Dense(units = num_classes , activation = 'sigmoid')(top_model)
  return top_model

In [17]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, GlobalAveragePooling2D, BatchNormalization, Dropout, Activation
from keras.layers import ZeroPadding2D

num_classes = 2

FC_Head = addTopModelMobileNet(MobileNet , num_classes)
model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0   

In [18]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [32]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/content/drive/My Drive/dataset/training_set/'
validation_data_dir =  '/content/drive/My Drive/dataset/test_set/'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [37]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("cat_vs_dog_mobileNet.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 1097
nb_validation_samples = 272

# We only train 5 EPOCHS 
epochs = 5
batch_size = 32

history = model.fit(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/5
34/34 [==============================] - 343s 10s/step - loss: 1.2949 - accuracy: 0.7024 - val_loss: 0.1348 - val_accuracy: 0.9453

Epoch 00001: val_loss improved from inf to 0.13484, saving model to cat_vs_dog_mobileNet.h5
Epoch 2/5
34/34 [==============================] - 289s 9s/step - loss: 0.2126 - accuracy: 0.9128 - val_loss: 0.0772 - val_accuracy: 0.9609

Epoch 00002: val_loss improved from 0.13484 to 0.07723, saving model to cat_vs_dog_mobileNet.h5
Epoch 3/5
34/34 [==============================] - 248s 7s/step - loss: 0.1880 - accuracy: 0.9213 - val_loss: 0.0326 - val_accuracy: 0.9844

Epoch 00003: val_loss improved from 0.07723 to 0.03265, saving model to cat_vs_dog_mobileNet.h5
Epoch 4/5
34/34 [==============================] - 213s 6s/step - loss: 0.2221 - accuracy: 0.9117 - val_loss: 0.0455 - val_accuracy: 0.9844

Epoch 00004: val_loss did not improve from 0.03265
Epoch 5/5
34/34 [==============================] - 182s 5s/step - loss: 0.1155 - accuracy: 0.9549 - 

In [38]:
from keras.models import load_model

classifier = load_model('cat_vs_dog_mobileNet.h5')

In [40]:
from keras.preprocessing import image
import numpy as np

In [49]:
test_image = image.load_img('download (2).jpg', target_size=(224,224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image , axis = 0)
result = classifier.predict(test_image)
result = np.argmax(result)
if result == 0:
  print("Cat")
else:
  print('Dog')


Cat


In [44]:
train_generator.class_indices

{'cats': 0, 'dogs': 1}